# Game Data Crawling

권상, 정윤

## Empty Table

In [1]:
import re
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.headless = True
driver = webdriver.Chrome('/Applications/chromedriver', options=options)

In [3]:
driver.get('https://www.koreabaseball.com/Schedule/Schedule.aspx')
driver.find_element_by_id('ddlMonth').send_keys('07')
time.sleep(1)
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

In [4]:
jul = pd.read_html(str(soup.select('div.tbl-type06')))[0].iloc[90:, :3].reset_index(drop=True)

In [5]:
a = pd.DataFrame(columns=['날짜', '시간', '경기'])
for i in range(jul.shape[0]):
    a = pd.concat([a, pd.DataFrame([jul.iloc[i, :]]), pd.DataFrame([jul.iloc[i, :]])])
jul = a.reset_index(drop=True)
jul = jul.assign(PA=0, AB=0, RBI=0, RUN=0, HIT=0, H2=0, H3=0, HR=0, SB=0, CS=0, SH=0, SF=0, BB=0, IB=0,
           HP=0, KK=0, GD=0, ERR=0, CG_CK=0, WLS=0, HOLD=0, WP=0, BK=0, INN2=0, BF=0, ER=0, R=0)

## Player Stat

In [6]:
options.headless = True
driver = webdriver.Chrome('/Applications/chromedriver', options=options)
driver.get('https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx')
driver.find_element_by_css_selector('#contents > div.date-select > ul > li.today > img').click()
driver.find_element_by_css_selector('#ui-datepicker-div > div > div > select.ui-datepicker-month').send_keys('7월(JUL)')
driver.find_element_by_css_selector('#ui-datepicker-div > table > tbody > tr:nth-child(4) > td:nth-child(2) > a').click()

n = 0

for j in range(11):
    for i in range(1, 6):
        try:
            driver.find_element_by_css_selector('#contents > div.today-game > div > div.bx-viewport > ul > li:nth-child(%d) > div > div.game-cont' % i).click()
            driver.find_element_by_css_selector('#tabDepth2 > li:nth-child(2) > a').click()
            
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            table = pd.read_html(str(soup.select('div.summary.tbl-type02.mt30.mb30')))[0]
            table.index = table[0]
            table = table.drop([0], axis=1)
            table = table.T.reset_index(drop=True)
            
            T_hit = pd.concat([pd.read_html(str(soup.select('div.hitter-record-area')))[0],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[1],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[2]], axis=1).iloc[:, 2:]
            
            B_hit = pd.concat([pd.read_html(str(soup.select('div.hitter-record-area')))[3],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[4],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[5]], axis=1).iloc[:, 2:]
            
            T_pit = pd.read_html(str(soup.select('div.pitcher-record-area')))[0]
            
            B_pit = pd.read_html(str(soup.select('div.pitcher-record-area')))[1]
         
            jul.iloc[n, 3] = int(T_hit.loc[T_hit['선수명'].isnull(), '타수'])
            jul.iloc[n, 5] = int(T_hit.loc[T_hit['선수명'].isnull(), '타점'])
            jul.iloc[n, 6] = int(T_hit.loc[T_hit['선수명'].isnull(), '득점'])
            jul.iloc[n, 7] = int(T_hit.loc[T_hit['선수명'].isnull(), '안타'])
            jul.iloc[n+1, 3] = int(B_hit.loc[B_hit['선수명'].isnull(), '타수'])
            jul.iloc[n+1, 5] = int(B_hit.loc[B_hit['선수명'].isnull(), '타점'])
            jul.iloc[n+1, 6] = int(B_hit.loc[B_hit['선수명'].isnull(), '득점'])
            jul.iloc[n+1, 7] = int(B_hit.loc[B_hit['선수명'].isnull(), '안타'])
            
            for i in range(T_hit.shape[0]):
                for j in range(1, 10):
                    if T_hit.iloc[i, j] == '4구':
                        jul.iloc[n, 15] += 1
                    elif T_hit.iloc[i, j] == '고4':
                        jul.iloc[n+1, 16] += 1
                    elif T_hit.iloc[i, j] == '사구':
                        jul.iloc[n+1, 17] += 1
                for j in range(1, 10):
                    if re.search('[가-힣]2', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('H2')] += 1
                    elif re.search('[가-힣]3', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('H3')] += 1
                    elif re.search('[가-힣]홈', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('HR')] += 1
                    elif re.search('[가-힣]희번', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('SH')] += 1
                    elif re.search('[가-힣]희비', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('SF')] += 1
                    elif re.search('[가-힣]병', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('GD')] += 1
            
            for i in range(B_hit.shape[0]):
                for j in range(1, 10):
                    if B_hit.iloc[i, j] == '4구':
                        jul.iloc[n+1, 15] += 1
                    elif B_hit.iloc[i, j] == '고4':
                        jul.iloc[n+1, 16] += 1
                    elif B_hit.iloc[i, j] == '사구':
                        jul.iloc[n+1, 17] += 1
                for j in range(1, 10):
                    if re.search('[가-힣]2', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('H2')] +=  1
                    elif re.search('[가-힣]3', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('H3')] +=  1
                    elif re.search('[가-힣]홈', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('HR')] += 1
                    elif re.search('[가-힣]희번', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('SH')] += 1
                    elif re.search('[가-힣]희비', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('SF')] += 1
                    elif re.search('[가-힣]병', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('GD')] += 1
            
            if '도루' in table.columns:
                for i in range(len(table['도루'][0].split(' '))):
                    if table['도루'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        jul.iloc[n, 11] += 1
                    elif table['도루'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        jul.iloc[n+1, 11] += 1
            else: pass
            
            if '도루자' in table.columns:
                for i in range(len(table['도루자'][0].split(' '))):
                    if table['도루자'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        jul.iloc[n, 12] = jul.iloc[n, 12] + 1
                    elif table['도루자'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        jul.iloc[n+1, 12] = jul.iloc[n+1, 12] + 1
            else: pass
            
            if '실책' in table.columns:
                for i in range(len(table['실책'][0].split(' '))):
                    if table['실책'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        jul.iloc[n, 20] += 1
                    elif table['실책'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        jul.iloc[n+1, 20] += 1
            
            # CG_CK
            if T_pit.shape[0] == 2:
                jul.iloc[n, 21] = 1
            else: pass
            if B_pit.shape[0] == 2:
                jul.iloc[n+1, 21] = 1
            else: pass
            
            if '승' in list(T_pit.iloc[:,2]):
                jul.iloc[n, 22] = 'W'
                jul.iloc[n+1, 22] = 'L'
            elif '패' in list(T_pit.iloc[:,2]):
                jul.iloc[n, 22] = 'L'
                jul.iloc[n+1, 22] = 'W'
            else:
                jul.iloc[n, 22] = 'D'
                jul.iloc[n+1, 22] = 'D'
            
            if '폭투' in table.columns:
                for i in range(len(table['폭투'][0].split(' '))):
                    if table['폭투'][0].split(' ')[i][:-4] in list(T_pit['선수명']):
                        jul.iloc[n, 24] += 1
                    elif table['폭투'][0].split(' ')[i][:-4] in list(B_pit['선수명']):
                        jul.iloc[n+1, 24] += 1
            
            if '보크' in table.columns:
                for i in range(len(table['보크'][0].split(' '))):
                    if table['보크'][0].split(' ')[i][:-4] in list(T_pit['선수명']):
                        jul.iloc[n, 25] += 1
                    elif table['보크'][0].split(' ')[i][:-4] in list(B_pit['선수명']):
                        jul.iloc[n+1, 25] += 1  
            
            if '홀드' in list(T_pit.iloc[:,2]):
                    jul.iloc[n, 23] = list(T_pit.iloc[:,2]).count('홀드')
            if '홀드' in list(B_pit.iloc[:,2]):
                jul.iloc[n+1, 23] = list(B_pit.iloc[:,2]).count('홀드')
            
            # KK
            jul.iloc[n, 18] = T_pit.iloc[-1, 13]
            jul.iloc[n+1, 18] = B_pit.iloc[-1, 13]
            
            # INN2
            jul.iloc[n, 26] = T_pit.iloc[-1, 6]
            jul.iloc[n+1, 26] = B_pit.iloc[-1, 6]
            
            # BF
            jul.iloc[n, 27] = T_pit.iloc[-1, 8]
            jul.iloc[n+1, 27] = B_pit.iloc[-1, 8]
            
            # ER
            jul.iloc[n, 28] = T_pit.iloc[-1, 15]
            jul.iloc[n+1, 28] = B_pit.iloc[-1, 15]
            
            # R
            jul.iloc[n, 29] = T_pit.iloc[-1, 14]
            jul.iloc[n+1, 29] = B_pit.iloc[-1, 14]
            
            # AB
            jul.iloc[n, 4] = T_pit.iloc[-1, 7]
            jul.iloc[n+1, 4] = B_pit.iloc[-1, 7]
            
        except: pass
        
        n += 2

    driver.find_element_by_css_selector('#lnkNext > img').click()

In [7]:
jul_a = jul.iloc[[2*n for n in range(51)],].reset_index(drop=True)
jul_b = jul.iloc[[2*n+1 for n in range(51)],].reset_index(drop=True)

In [8]:
jul_a = jul_a.assign(T=None, B=None)
jul_b = jul_b.assign(T=None, B=None)

In [9]:
for i in range(jul_a.shape[0]):
    jul_a.iloc[i, 30] = jul_a.iloc[i, 2].split('vs')[0][:2]
    jul_a.iloc[i, 31] = jul_a.iloc[i, 2].split('vs')[1][-2:]
for i in range(jul_b.shape[0]):
    jul_b.iloc[i, 30] = jul_b.iloc[i, 2].split('vs')[0][:2]
    jul_b.iloc[i, 31] = jul_b.iloc[i, 2].split('vs')[1][-2:]

In [10]:
for i in range(jul_a.shape[0]):
    jul_a.iloc[i, 0] = '2020.' + jul_a.iloc[i, 0][:5]
    jul_a.iloc[i, 0] = jul_a.iloc[i, 0].replace('.', '')
for i in range(jul_b.shape[0]):
    jul_b.iloc[i, 0] = '2020.' + jul_b.iloc[i, 0][:5]
    jul_b.iloc[i, 0] = jul_b.iloc[i, 0].replace('.', '')

In [11]:
name = {'NC':'NC', '두산':'OB', '키움':'WO', 'KI':'HT', 'IA':'HT', 'LG':'LG',
       '삼성':'SS', 'KT':'KT', '롯데':'LT', 'SK':'SK', '한화':'HH'}

f = lambda x: name.get(x, x)
jul_a['T'] = jul_a['T'].map(f) ; jul_a['B'] = jul_a['B'].map(f)
jul_b['T'] = jul_b['T'].map(f) ; jul_b['B'] = jul_b['B'].map(f)

In [12]:
jul_a['G_ID'] = jul_a['날짜'] + jul_a['T'] + jul_a['B'] + '0'
jul_b['G_ID'] = jul_b['날짜'] + jul_b['T'] + jul_b['B'] + '1'

In [13]:
jul_a = jul_a.drop(['시간', '경기'], axis=1)
jul_b = jul_b.drop(['시간', '경기'], axis=1)

In [14]:
jul_a = jul_a.rename(columns={'T':'T_ID', 'B':'VS_T_ID'})
jul_b = jul_b.rename(columns={'T':'VS_T_ID', 'B':'T_ID'})

In [15]:
jul_a = jul_a.assign(GDAY_DS=None, HEADER_NO=None, TB_SC=None)
jul_b = jul_b.assign(GDAY_DS=None, HEADER_NO=None, TB_SC=None)

In [16]:
for i in range(jul_a.shape[0]):
    jul_a.iloc[i, 31] = jul_a.iloc[i, 30][:8]
    jul_a.iloc[i, 32] = jul_a.iloc[i, 30][12]
    jul_a.iloc[i, 33] = 'T'
for i in range(jul_b.shape[0]):
    jul_b.iloc[i, 31] = jul_b.iloc[i, 30][:8]
    jul_b.iloc[i, 32] = jul_b.iloc[i, 30][12]
    jul_b.iloc[i, 33] = 'B'

In [17]:
jul = pd.concat([jul_a, jul_b]).sort_values('G_ID').reset_index(drop=True)

In [18]:
jul_hit = jul[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'PA', 'AB',
       'RBI', 'RUN', 'HIT', 'H2', 'H3', 'HR', 'SB', 'CS', 'SH', 'SF', 'BB',
       'IB', 'HP', 'KK', 'GD', 'ERR']]
jul_hit = jul_hit.assign(LOB=None, P_HRA_RT=None, P_AB_CN=None, P_HIT_CN=None)

In [19]:
jul_hit.head()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,PA,AB,RBI,RUN,...,BB,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN
0,20200721HTHH0,20200721,HT,HH,0,T,41,32,9,10,...,8,0,0,7,2,0,None,None,None,None
1,20200721HTHH1,20200721,HH,HT,1,B,29,50,2,2,...,2,0,2,8,1,3,None,None,None,None
2,20200721LGKT0,20200721,LG,KT,0,T,38,39,9,9,...,6,0,0,7,0,0,None,None,None,None
3,20200721LGKT1,20200721,KT,LG,1,B,34,44,10,10,...,4,0,0,7,0,1,None,None,None,None
4,20200721LTSK0,20200721,LT,SK,0,T,35,40,7,7,...,5,0,0,6,1,0,None,None,None,None


In [20]:
jul_pit = jul[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'CG_CK',
       'WLS', 'HOLD', 'INN2', 'BF', 'PA', 'AB', 'HIT', 'H2', 'H3', 'HR', 'SB',
       'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'WP', 'BK', 'ERR', 'R',
       'ER']]

jul_pit = jul_pit.assign(P_WHIP_RT=None, P2_WHIP_RT=None, CB_WHIP_RT=None)

In [21]:
jul_pit['INN2'] = jul_pit['INN2'].astype('str')

In [22]:
for i in range(jul_pit.shape[0]):
    if len(jul_pit.iloc[i, 9].split(' ')) == 1:
        jul_pit.iloc[i, 9] = int(jul_pit.iloc[i, 9])*3
    else:
        jul_pit.iloc[i, 9] = int(jul_pit.iloc[i, 9].split(' ')[0])*3 + (int(jul_pit.iloc[i, 9].split(' ')[1].split('/')[0])/int(jul_pit.iloc[i, 9].split(' ')[1].split('/')[1]))*3

In [23]:
jul_pit.head()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,CG_CK,WLS,HOLD,INN2,...,KK,GD,WP,BK,ERR,R,ER,P_WHIP_RT,P2_WHIP_RT,CB_WHIP_RT
0,20200721HTHH0,20200721,HT,HH,0,T,0,W,0,27,...,7,2,0,0,0,2,2,None,None,None
1,20200721HTHH1,20200721,HH,HT,1,B,0,L,0,27,...,8,1,2,0,3,10,9,None,None,None
2,20200721LGKT0,20200721,LG,KT,0,T,0,L,0,24,...,7,0,1,1,0,10,10,None,None,None
3,20200721LGKT1,20200721,KT,LG,1,B,0,W,1,27,...,7,0,1,0,1,9,9,None,None,None
4,20200721LTSK0,20200721,LT,SK,0,T,0,L,2,25,...,6,1,0,0,0,8,8,None,None,None


## Game Schedule Crawling

**September**

In [24]:
driver.get('https://www.koreabaseball.com/Schedule/Schedule.aspx')
driver.find_element_by_id('ddlMonth').send_keys('09')

req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

In [25]:
schedule_sep = pd.read_html(str(soup.select('div.tbl-type06')))[0].iloc[127:, :3]
schedule_sep

,날짜,시간,경기
127,09.29(화),18:30,롯데vsLG
128,09.29(화),18:30,KTvs삼성
129,09.29(화),18:30,SKvsNC
130,09.29(화),18:30,KIAvs키움
131,09.29(화),18:30,두산vs한화
132,09.30(수),14:00,롯데vsLG
133,09.30(수),14:00,KTvs삼성
134,09.30(수),14:00,SKvsNC
135,09.30(수),14:00,KIAvs키움
136,09.30(수),14:00,두산vs한화


**October**

In [26]:
driver.get('https://www.koreabaseball.com/Schedule/Schedule.aspx')
driver.find_element_by_id('ddlMonth').send_keys('10')

req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

In [27]:
schedule_oct = pd.read_html(str(soup.select('div.tbl-type06')))[0].iloc[:, :3]
schedule_oct

,날짜,시간,경기
0,10.01(목),14:00,롯데vsLG
1,10.01(목),14:00,KTvs삼성
2,10.01(목),14:00,SKvsNC
3,10.01(목),14:00,KIAvs키움
4,10.01(목),14:00,두산vs한화
...,...,...,...
80,10.18(일),14:00,KIAvsLG
81,10.18(일),14:00,KTvsSK
82,10.18(일),14:00,롯데vsNC
83,10.18(일),14:00,두산vs키움


**Total**

In [28]:
schedule = pd.concat([schedule_sep, schedule_oct]).reset_index(drop=True)

In [29]:
schedule['T'] = None
schedule['B'] = None

In [30]:
for i in range(schedule.shape[0]):
    schedule.iloc[i, 3] = schedule.iloc[i, 2].split('vs')[0]
    schedule.iloc[i, 4] = schedule.iloc[i, 2].split('vs')[1]

In [31]:
for i in range(schedule.shape[0]):
    schedule.iloc[i, 0] = '2020.' + schedule.iloc[i, 0][:5]
    schedule.iloc[i, 0] = schedule.iloc[i, 0].replace('.', '')

In [32]:
name = {'NC':'NC', '두산':'OB', '키움':'WO', 'KIA':'HT', 'LG':'LG',
       '삼성':'SS', 'KT':'KT', '롯데':'LT', 'SK':'SK', '한화':'HH'}

f = lambda x: name.get(x, x)
schedule['T'] = schedule['T'].map(f)
schedule['B'] = schedule['B'].map(f)

In [33]:
schedule = pd.DataFrame(schedule['날짜'] + schedule['T'] + schedule['B'], columns=['G_ID'])

In [34]:
for i in schedule[schedule.duplicated(['G_ID'], keep=False) == True].index:
    schedule.iloc[i, 0] = schedule.iloc[i, 0] + '1'
for i in schedule[schedule.duplicated(['G_ID'], keep=False) == False].index:
    schedule.iloc[i, 0] = schedule.iloc[i, 0] + '0'

In [35]:
schedule['GDAY_DS'] = None
schedule['T_ID'] = None
schedule['VS_T_ID'] = None
schedule['HEADER_NO'] = None
schedule['TB_SC'] = None

In [36]:
schedule_B = schedule.copy()

In [37]:
for i in range(schedule.shape[0]):
    schedule.iloc[i, 1] = schedule.iloc[i, 0][:8]
    schedule.iloc[i, 2] = schedule.iloc[i, 0][8:10]
    schedule.iloc[i, 3] = schedule.iloc[i, 0][10:12]
    schedule.iloc[i, 4] = schedule.iloc[i, 0][12]
    schedule.iloc[i, 5] = 'T'
for i in range(schedule_B.shape[0]):
    schedule_B.iloc[i, 1] = schedule_B.iloc[i, 0][:8]
    schedule_B.iloc[i, 2] = schedule_B.iloc[i, 0][10:12]
    schedule_B.iloc[i, 3] = schedule_B.iloc[i, 0][8:10]
    schedule_B.iloc[i, 4] = schedule_B.iloc[i, 0][12]
    schedule_B.iloc[i, 5] = 'B'

In [38]:
schedule = pd.concat([schedule, schedule_B]).sort_values(['GDAY_DS', 'T_ID']).reset_index(drop=True)

In [39]:
schedule

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC
0,20200929OBHH0,20200929,HH,OB,0,B
1,20200929HTWO0,20200929,HT,WO,0,T
2,20200929KTSS0,20200929,KT,SS,0,T
3,20200929LTLG0,20200929,LG,LT,0,B
4,20200929LTLG0,20200929,LT,LG,0,T
...,...,...,...,...,...,...
185,20201018LTNC0,20201018,NC,LT,0,B
186,20201018OBWO0,20201018,OB,WO,0,T
187,20201018KTSK0,20201018,SK,KT,0,B
188,20201018SSHH0,20201018,SS,HH,0,T
